In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'1.14.0'

Some layers, in particular the `BatchNormalization` layer and the `Dropout` layer, have different behaviors during training and inference. For such layers, it is standard practice to expose a `training` (boolean) argument in the `call` method.

By exposing this argument in `call`, you enable the built-in training and evaluation loops to correctly use the layer in training and inference.

In [2]:
class Wrapper(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Wrapper, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.dropout = tf.keras.layers.Dropout(0.9)
        super(Wrapper, self).build(input_shape)
        
    def call(self, inputs, training=True):
        x = self.dropout(inputs, training=training)
        return x

In [3]:
def make_model():
    inputs = tf.keras.Input(shape=(10,))
    outputs = Wrapper()(inputs)
    model = tf.keras.Model(inputs, outputs)
    return model

model = make_model()

W1214 20:28:02.196884 19908 nn_ops.py:4224] Large dropout rate: 0.9 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [4]:
drop = model(model.inputs, training=True)
no_drop = model(model.inputs, training=False)

W1214 20:28:02.786490 19908 nn_ops.py:4224] Large dropout rate: 0.9 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [5]:
with tf.Session() as sess:
    print(sess.run(drop, feed_dict={model.input: np.ones((1,10))}))
    print(sess.run(no_drop, feed_dict={model.input: np.ones((1,10))}))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
